# Induced gap and magnetic field

# TO-DO:
* Remove the use of `delta` function in `tunnel_hoppings`
* Correct for zero supercurrent because of non-zero total $A$ in SC, example from [orbitalfield](https://github.com/basnijholt/orbitalfield) project:

```python
    def peierls(val, ind, a):
        def phase(s1, s2, p):
            x, y, z = s1.pos
            A_site = [p.B_y * z - p.B_z * y, 0, p.B_x * y][ind]
            if p.A_correction:
                A_sc = [A(p, *site.pos) for site in sc_sites]
                A_site -= np.mean(A_sc, axis=0)[ind]
            A_site *= a * 1e-18 * eV / hbar
            return np.cos(A_site) * s0s0 - 1j * np.sin(A_site) * s0sz
```

In [ ]:
# 1. Standard library imports
from types import SimpleNamespace
from importlib import reload
from itertools import product

# 2. External package imports
import holoviews as hv
import kwant
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy

# 3. Internal imports
import funcs
reload(funcs)

hv.notebook_extension('bokeh')
print(kwant.__version__)

# Usage

In [ ]:
ham_pars = dict(alpha=20, B_x=0, B_y=0, B_z=0, Delta=0.250, g=50, mu_B=funcs.constants.mu_B,
                orbital=True, c=funcs.constants.c, hbar=funcs.constants.hbar,
                m_eff=funcs.constants.m_eff, mu_sc=10, mu_sm=10, e=funcs.constants.e, c_tunnel=1, V=0)

syst_pars = dict(a=10, angle=45, onsite_disorder=False,
                   L=200, phi=135, r1=50, r2=80, shape='circle',
                   with_leads=True, with_shell=True)

syst = funcs.make_3d_wire(**syst_pars)
kwant.plot(syst)

In [ ]:
%%opts Path [aspect='square']
lead = funcs.make_lead(10, 50, 70, 135, 0, True, 'circle')
ham_pars['B_x'] = 1
ham_pars['B_y'] = 1
ham_pars['orbital'] = True
ks = np.linspace(-3, 3)
Es = funcs.bands(lead, ham_pars, ks)
p1 = hv.Path((ks, Es))[:, -100:100]

ham_pars['orbital'] = False
Es1 = funcs.bands(lead, ham_pars, ks)
p2 = hv.Path((ks, Es1))[:, -100:100]

p1 * p2

In [ ]:
%%time
gap = funcs.find_gap(lead, ham_pars)
print('The bandgap is {} meV'.format(gap))

In [ ]:
ham_pars = dict(alpha=20, B_x=0, B_y=0, B_z=0, Delta=60, g=50, mu_B=funcs.constants.mu_B,
                orbital=True, c=funcs.constants.c, hbar=funcs.constants.hbar,
                m_eff=funcs.constants.m_e, mu_sc=5, mu_sm=5, e=funcs.constants.e, c_tunnel=7/8, V=0)

S = kwant.smatrix(syst, params=ham_pars)

In [ ]:
funcs.andreev_conductance(syst, ham_pars)

# Dask

In [ ]:
import dask
import distributed
from dask import delayed

client = distributed.Client()
client.upload_file('funcs.py')
Bs = np.linspace(0, 1, 5)
biases = np.linspace(-100, 100, 5)
vals = list(product(Bs, biases))
gaps = []
for B_x, E_bias in vals:
    lead = delayed(funcs.make_lead)(10, 50, 70, 135, 0, True, 'circle')
    ham_pars['B_x'] = B_x
    ham_pars['V'] = E_bias
    gap = delayed(funcs.find_gap)(lead, ham_pars)
    gaps.append(gap)

l = client.compute(gaps)

distributed.progress(l)

res = client.gather(l)

# ipyparallel

In [ ]:
# Make sure this folder is in your $PYTHONPATH
import socket
if socket.gethostname() == 'hpc05':
    from ipyparallel import Client
    client = Client(profile='pbs')
else:
    try:
        # If this can be imported, it assumes you are on the TU Delft network with access to cluster
        import hpc051
        client = hpc05.Client(profile='pbs', timeout=60)
        print("Connected to hpc05")
    except ImportError:
        from ipyparallel import Client
        client = Client()
        print("Connected to local engines")

In [ ]:
dview = client[:]
dview.use_dill()
lview = client.load_balanced_view()
print(len(dview))
%px import sys, os; sys.path.append(os.path.expanduser('~/Work/induced_gap_B_field/'))

In [ ]:
ham_pars = dict(alpha=20, B_y=0, B_z=0, Delta=0.250, g=50, mu_B=funcs.constants.mu_B,
                orbital=True, c=funcs.constants.c, hbar=funcs.constants.hbar,
                m_eff=funcs.constants.m_eff, mu_sc=10, mu_sm=10, e=funcs.constants.e, c_tunnel=1)

syst_pars = dict(a=10, angle=45, phi=135, r1=50, r2=80, shape='circle', with_shell=True)

def gap(val, syst_pars=syst_pars, ham_pars=ham_pars):
    import funcs, socket
    ham_pars['B_x'], ham_pars['V'] = val
    lead = funcs.make_lead(**syst_pars)
    return {'E_gap': funcs.find_gap(lead, ham_pars), 'hostname': socket.gethostname()}

Bs = np.linspace(0, 1, 50)
Vs = np.linspace(-30, 30, 50)
vals = list(product(Bs, Vs))

E_gap = lview.map_async(gap, vals)
E_gap.wait_interactive()
E_gap = E_gap.result()

df = pd.DataFrame(vals, columns=['B_x', 'V'])
df = pd.concat((pd.DataFrame(E_gap), df), axis=1)
df = df.assign(**syst_pars).assign(**ham_pars).assign(**funcs.constants.__dict__)
df = df.assign(git_hash=funcs.get_git_revision_hash())
df.to_hdf('test.hdf', 'all_data', mode='w')

In [ ]:
da = df.set_index(['B_x', 'V']).to_xarray()
ds = hv.Dataset(da)
ds.to(hv.GridImage, kdims=['B_x', 'V'], vdims=['E_gap'])